In [5]:
import os
import torch
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer


In [6]:
# ----------- Setting up the stage ----------
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
VECTOR_DB_DIR = os.path.join(DATA_DIR, "chroma_db") # Directory where ChromaDB is persisted

COLLECTION_NAME = "pakistan_laws_chunks_collection"
EMBEDDING_MODEL_NAME = "BAAI/bge-large-en-v1.5" # This MUST be the exact model you used to generate embeddings

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Vector DB directory: {VECTOR_DB_DIR}")

Project root: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG
Data directory: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG/data
Vector DB directory: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG/data/chroma_db


In [7]:
# ----- Setting up my love with chromadb via its client ------
print(f"\nInitializing ChromaDB client at: {VECTOR_DB_DIR}")
try:
    client = chromadb.PersistentClient(path=VECTOR_DB_DIR)
    embedding_function_for_chroma = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=EMBEDDING_MODEL_NAME,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    collection = client.get_collection(
        name = COLLECTION_NAME,
        embedding_function = embedding_function_for_chroma
    )
    print(f"Successfully connected to ChromaDB collection '{COLLECTION_NAME}'.")
    print(f"Current count in collection: {collection.count()} chunks.")

except Exception as e:
    print(f"Error connecting to ChromaDB collection: {e}")
    print("Please ensure your 'chroma_db' directory exists and contains data.")
    print("Also verify 'COLLECTION_NAME' and 'EMBEDDING_MODEL_NAME' are correct.")
    # Exit or handle error appropriately if connection fails
    exit()
    


Initializing ChromaDB client at: /Users/apple/PycharmProjects/LLM_Production_01_RAG/Haq_ooq_RAG/data/chroma_db
Successfully connected to ChromaDB collection 'pakistan_laws_chunks_collection'.
Current count in collection: 30122 chunks.


In [8]:
query_embedding_model = None

try:
    query_embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nQuery embedding model '{EMBEDDING_MODEL_NAME}' loaded successfully.")
    
except Exception as e:
    print(f"Error loading query embedding model: {e}")
    print("Please ensure the model can be downloaded/loaded. Exiting for now.")
    exit() # Or handle more gracefully


Query embedding model 'BAAI/bge-large-en-v1.5' loaded successfully.


In [9]:
# --- lets engineer the Retriever Function ---

def retrieve_relevant_chunks(query_text: str, n_results: int = 5) -> list:
    """
    Embeds the query and retrieves the most relevant chunks from ChromaDB.

    Args:
        query_text (str): The user's query.
        n_results (int): The number of top relevant chunks to retrieve.

    Returns:
        list: A list of dictionaries, where each dictionary represents a retrieved chunk
              with its content and metadata.
    """
    if query_embedding_model is None:
        print("Query embedding model not loaded. Cannot retrieve chunks.")
        return []

    # Generate embedding for the query using the same model
    query_embedding = query_embedding_model.encode(query_text).tolist()
    
    # Query the ChromaDB collection
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=['documents', 'metadatas', 'distances'] # Request documents, their metadata, and similarity scores
    )
    retrieved_chunks_info = []
    
    if results and results['documents']:
        # Results are lists of lists, so we take the first element (for our single query)
        for i in range(len(results['documents'][0])):
            chunk_content = results['documents'][0][i]
            metadata = results['metadatas'][0][i]
            distance = results['distances'][0][i] # Similarity score (lower is more similar)

            retrieved_chunks_info.append({
                "chunk_content": chunk_content,
                "source_file": metadata.get('source_file'),
                "section_title": metadata.get('section_title'),
                "distance": distance # Include distance for debugging/understanding relevance
            })
    
    print(f"\nRetrieved {len(retrieved_chunks_info)} chunks for the query: '{query_text}'")
    return retrieved_chunks_info

# Defining the Tools

In [10]:
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

In [11]:
# Local Legal Document Search Tool
@tool
def legal_document_search(query: str) -> str:
    """Searches the local legal documents for relevant information.
    Use this tool when the question is about specific Pakistani laws, ordinances, or legal documents that might be in the local knowledge base.
    Input should be a clear, standalone question or keyword phrase relevant to the local documents."""
    print(f"\n--- Using legal_document_search tool for query: '{query}' ---")
    chunks = retrieve_relevant_chunks(query, n_results=5)
    if not chunks:
        return "No relevant documents found in local knowledge base!"
    formatted_context = "\n\n".join(chunk['chunk_content'] for chunk in chunks)
    return formatted_context


# Web Search Tool
web_search_tool = DuckDuckGoSearchRun(
    name="web_search",
    description="Useful for general knowledge questions or when information is not found in local legal documents. Input should be a concise search query."
)


tools = [legal_document_search, web_search_tool]
print("Tools defined: legal_document_search, web_search")



Tools defined: legal_document_search, web_search


# Setting up the LLM agent to use our tools to answer user's question

In [12]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama


In [14]:
try:
    llm = ChatOllama(model="qwen3:1.7b") # <<< Ensure 'model=' is used here
    print("Generative LLM (qwen3:1.7b via Ollama) initialized successfully.")
except Exception as e:
    print(f"Error initializing Ollama LLM: {e}")
    print("Please ensure Ollama is installed and the 'qwen3:1.7b' model is downloaded and running.")
    print("You might need to start Ollama with 'ollama run qwen3:1.7b' in your terminal.")
    exit() # Or handle error more gracefully
    
    
# Define the Agent Prompt
# This prompt guides the LLM on its role and how to use the tools.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful legal assistant specializing in Pakistani law. "
               "Your primary goal is to answer questions related to Pakistani laws, ordinances, and legal documents. "
               "You have access to two specialized tools:\n"
               "1. `legal_document_search`: Use this tool first for **any question that appears to be about Pakistani laws or legal documents** within your local knowledge base. You **MUST** call this tool and process its full output before deciding the next step.\n"
               "2. `web_search`: Use this tool *only* if the question is explicitly related to **Pakistani laws or legal affairs** (e.g., current legal officeholders, recent amendments to laws not in your local documents, or clarification on a legal term/concept where local docs are insufficient) AND the `legal_document_search` tool's *actual output* did not provide a comprehensive, up-to-date, or specific answer required by the query. You **MUST** call this tool and process its full output before proceeding with an answer if it's deemed necessary.\n\n"
               "**CRITICAL EXECUTION GUIDELINES:**\n"
               "- **NEVER** provide an answer to a question that requires tool use without **FIRST** explicitly invoking the necessary tool(s) and carefully reviewing their *actual, complete output*.\n"
               "- **DO NOT rely on your internal knowledge for current events, specific names (like current officeholders), or any dynamic information.** If such information is needed for a legal question, you **MUST** use `web_search` if `legal_document_search` fails to provide it. You *cannot* assume you know the answer or make it up.\n"
               "- If a question is **NOT related to Pakistani laws or legal affairs at all** (e.g., general knowledge, economic data like inflation, sports scores, non-legal biographical information), **DO NOT use ANY search tools (`legal_document_search` or `web_search`)**. Instead, politely inform the user that you are a specialized legal assistant and cannot provide information on that non-legal topic, suggesting they consult a more appropriate source.\n"
               "- **Every answer you provide MUST be directly supported by the content retrieved from the invoked tool(s).** If a tool's output is empty or does not contain the specific answer needed, you **MUST NOT** make up information or attribute an answer to that tool.\n"
               "- If no relevant information is found after *actually invoking and reviewing the outputs* of all appropriate tools (based on the question type and the real content from tool outputs), then state that you cannot find the answer.\n"
               "- When providing an answer, be precise and **cite the exact source tool and any relevant document names or dates** (e.g., 'According to the local legal documents...', 'A web search indicates...', 'Based on the legal_document_search results from The Privatisation Commission Ordinance, 2000...')."),
    ("user", "{question}"),
    ("placeholder", "{agent_scratchpad}") # This is for the agent's internal thought process and tool calls
])

Generative LLM (qwen3:1.7b via Ollama) initialized successfully.


In [15]:
llm_with_tools = llm.bind_tools(tools)
agent = create_tool_calling_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
print("\nLLM Agent (with local and web search tools) assembled.")


LLM Agent (with local and web search tools) assembled.


In [29]:
# --- Test the Full RAG Pipeline with Agent ---
    
# Sample legal query that should use local documents
question_1 = "What are the functions and powers of the Privatisation Commission as per the Ordinance?"
print(f"\nQuestion 1: {question_1}")
response_1 = agent_executor.invoke({"question": question_1})
print("\nAnswer 1:")
print(response_1["output"]) # AgentExecutor returns a dict, output is in 'output' key



Question 1: What are the functions and powers of the Privatisation Commission as per the Ordinance?


> Entering new AgentExecutor chain...

Invoking: `legal_document_search` with `{'query': 'Functions and powers of Privatisation Commission as per Ordinance'}`
responded: <think>
Okay, the user is asking about the functions and powers of the Privatisation Commission as per the Ordinance. First, I need to check if there's a specific ordinance related to this. Since the user mentioned "Ordinance," I should use the legal_document_search tool first. The query should be about the functions and powers of the Privatisation Commission. Let me call the legal_document_search with the query "Functions and powers of Privatisation Commission as per Ordinance." 

If the search returns relevant information, I can extract the details. If not, I'll use the web_search tool for general knowledge. But since the user specifically mentioned an ordinance, I should prioritize the legal_document_search. Let me

/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `enco


Retrieved 5 chunks for the query: 'Functions and powers of Privatisation Commission as per Ordinance'
THE PRIVATISATION COMMISSION ORDINANCE, 2000 PART I.—GENERAL SECTIONS: 1. Short title, extent and commencement. 2. Definitions. PART II.—PRIVATISATION COMMISSION 3. Establishment of the Commission. 4. Location of Office. 5. Functions and Powers of the Commission. PART III. —MANAGEMENT AND ADMINISTRATION 6. Board of the Commission. 7. Chairman, Secretary and members. 8. Meetings of the Board. 9. Delegation. 10. Employees of the Commission. 11. Employment of agents, advisers and consultants. 12. Public Servants. 13. Disclosure of interest. PART IV. —FINANCIAL PROVISIONS 14. Funds of the Commission. 15. Expenditure to be charged on the Commission Account. 16. Privatisation Fund. 17. Power to obtain finance and r eceive grants. 18. Investment. 19. Bank accounts. 20. Accounts. 21. Audit. PART V. —PRIVATISATION 22. Privatisation programme. 23. Advertisement of privatisation. 24. Valuation o

/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/langchain_core/messages/ai.py:203: ResourceWarning: unclosed <socket.socket fd=81, family=2, type=1, proto=6, laddr=('127.0.0.1', 55231), raddr=('127.0.0.1', 11434)>
  check_additional_kwargs = not any(


<think>
Okay, the user asked about the functions and powers of the Privatisation Commission as per the Ordinance. I used the legal_document_search tool to find the relevant provisions. The response from the tool provided a detailed list of sections under the Ordinance, including functions like recommending privatization policies, preparing the privatization program, managing implementation, and publicizing activities. The user might need a clear summary of these functions. I should structure the answer to highlight the key functions and ensure clarity. Also, mention the specific sections from the Ordinance cited. Since the user is likely looking for a concise overview, I'll present the main functions in bullet points and reference the relevant parts of the Ordinance.
</think>

The Privatisation Commission, as outlined in **Section 5** of the **Privatization Commission Ordinance, 2000**, has the following key functions and powers:

1. **Policy Recommendations**:  
   - Recommend privati

In [30]:
# Sample query that should trigger a web search
question_2 = "What is the current inflation rate in Pakistan as of today?"
print(f"\n\nQuestion 2: {question_2}")
response_2 = agent_executor.invoke({"question": question_2})
print("\nAnswer 2:")
print(response_2["output"])



Question 2: What is the current inflation rate in Pakistan as of today?


> Entering new AgentExecutor chain...
<think>
The user is asking about the current inflation rate in Pakistan. This is a general knowledge question related to economic data, not specifically about Pakistani laws or legal documents. Since the assistant is specialized in legal matters, it should not use the web_search tool for this query. The assistant should inform the user that it cannot provide this type of information and suggest consulting a more appropriate source.
</think>

I am a specialized legal assistant and cannot provide information on economic data such as inflation rates. For this query, you may need to consult a financial or economic source, or check the official statistics provided by the Pakistan Bureau of Statistics.

> Finished chain.

Answer 2:
<think>
The user is asking about the current inflation rate in Pakistan. This is a general knowledge question related to economic data, not specifical

In [16]:
# Another sample query, perhaps one not directly answerable by the documents, prompting web search
question_3 = "Who is the current Chief Justice of Pakistan?"
print(f"\n\nQuestion 3: {question_3}")
response_3 = agent_executor.invoke({"question": question_3})
print("\nAnswer 3:")
print(response_3["output"]) # AgentExecutor returns a dict, output is in 'output' key



Question 3: Who is the current Chief Justice of Pakistan?


> Entering new AgentExecutor chain...

Invoking: `legal_document_search` with `{'query': 'current Chief Justice of Pakistan'}`
responded: <think>
Okay, the user is asking about the current Chief Justice of Pakistan. Let me check the tools available. The first tool is legal_document_search, which is for Pakistani laws and documents. The second is web_search for general knowledge. Since this is a legal question about a current officeholder, I should use legal_document_search first. But wait, the user is asking for the current Chief Justice, which might be covered in the legal documents. However, the local knowledge base might not have real-time data. If the legal_document_search doesn't provide the answer, then I need to use web_search. But according to the guidelines, I must first call legal_document_search. Let me try that. The query would be about the current Chief Justice. If the tool returns the information, I can use it.

/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Retrieved 5 chunks for the query: 'current Chief Justice of Pakistan'
_________, do solemnly swear that I will bear true faith and allegiance to Pakistan : That, as the Chief Justice of the Federal Shariat Court (or a Judge of the Federal Shariat Court), I will discharge my duties, and perform my functions, honestly, to the best of my ability , and faithfully, in accordance with the Constitution of the Islamic Republic of Pakistan and the law : That I will not allow my personal interest to influence my official conduct or my official decisions : That I will abide by the c ode of conduct issued by the Supreme Judicial Council : That I will preserve, protect and defend the Constitution of the Islamic Republic of Pakistan : And that , in all circumstances, I will do right to all manner of people, according to law, without f ear or favour, affection or ill -will. May Allah Almighty help and guide me (A’meen).] ___________ 1 Subs. by the Constitution (Eighteenth Amdt.) Act, 2010 (10 of 201

/Users/apple/PycharmProjects/LLM_Production_01_RAG/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


The chief justice of Pakistan (initials as CJP; Urdu: منصفِ اعظم پاکستان, Munsif-e-Āzam Pākistān) is the chief judge of the Supreme Court of Pakistan and is the … Oct 26, 2024 · Justice Yahya Afridi has been appointed new Chief Justice of Pakistan with effect from October 26, 2024. Justice Yahya Afridi was sworn … Oct 26, 2024 · Justice Yahya Afridi was sworn in as the 30th Chief Justice of Pakistan (CJP) on Saturday during an oath-taking ceremony at President … Justice Yahya Afridi is the current Chief Justice of Pakistan. He assumed office on October 26, 2024. He was appointed as a Chief Justice by Pakistani President … Oct 22, 2024 · Under the 26th Constitutional Amendment, the parliamentary committee on Tuesday nominated Justice Yahya Afridi as the …<think>
Okay, the user asked for the current Chief Justice of Pakistan. The previous response from the legal_document_search tool didn't provide the answer, so I used web_search. The web_search result mentions Justice Yahya Afridi takin